# Thư viện cần thiết

In [5]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, RandomizedSearchCV
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import RidgeCV

# Load dữ liệu

In [6]:
X_train = pd.read_csv('../exps/Preproccessed/exp1_train.csv')
X_test = pd.read_csv('../exps/Preproccessed/exp1_test.csv')
y = pd.read_csv('../data/train.csv')
y = y['Class']

# Tối ưu hóa, Huấn luyện và test

In [7]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse_scorer = make_scorer(rmse, greater_is_better=False)

## Model mặc định:

In [8]:
svc = SVC(probability=True)
rfc = RandomForestClassifier()
gbm = GradientBoostingClassifier()

In [9]:
# for name, model in tqdm(models, desc="Đang huấn luyện các model"):
#     results = {}
#     for name, model in models:
#         acc = cross_val_score(model, X_train, y, cv=kf, scoring='accuracy')
#         acc = acc.mean()
#         f1 = cross_val_score(model, X_train, y, cv=kf, scoring='f1_macro')
#         f1 = f1.mean()
#         auc = cross_val_score(model, X_train, y, cv=kf, scoring='roc_auc_ovr')
#         auc = auc.mean()

#         print(f"{name} CV Results:")
#         print(f"  Accuracy: {acc:.4f}")
#         print(f"  F1 Score: {f1:.4f}")
#         print(f"  ROC AUC:  {auc:.4f}")
#         print("-" * 35)

#cross-validate cho phép tính cùng lúc nhiều chỉ số chỉ trong 1 lần train
from tqdm import tqdm #hiển thị tiến trình
models = [('SVC', svc), ('RandomForest', rfc), ('GradientBoosting', gbm)]

scoring = {
    'accuracy': 'accuracy',
    'f1_macro': 'f1_macro',
    'roc_auc_ovr': 'roc_auc_ovr'
}

for name, model in tqdm(models):
    cv_results = cross_validate(model,X_train,y,cv=kf,scoring=scoring,
                                n_jobs=-1,verbose=1)
    
    acc = cv_results['test_accuracy'].mean()
    f1 = cv_results['test_f1_macro'].mean()
    auc = cv_results['test_roc_auc_ovr'].mean()

    print(f"\n{name} CV Results:")
    print(f"  Accuracy: {acc:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  ROC AUC:  {auc:.4f}")
    print("-" * 35)

  0%|          | 0/3 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  6.7min remaining: 10.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  6.8min finished
 33%|███▎      | 1/3 [06:48<13:37, 408.65s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.



SVC CV Results:
  Accuracy: 0.2748
  F1 Score: 0.0395
  ROC AUC:  0.4992
-----------------------------------


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    9.5s remaining:   14.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.7s finished
 67%|██████▋   | 2/3 [06:58<02:54, 174.09s/it]


RandomForest CV Results:
  Accuracy: 0.2149
  F1 Score: 0.0838
  ROC AUC:  0.5020
-----------------------------------


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.1min finished
100%|██████████| 3/3 [08:06<00:00, 162.11s/it]


GradientBoosting CV Results:
  Accuracy: 0.2596
  F1 Score: 0.0544
  ROC AUC:  0.5008
-----------------------------------


## Tối ưu hóa mô hình:

In [10]:
# === TỐI ƯU HÓA MÔ HÌNH VỚI RANDOMIZEDSEARCHCV ===

def random_search(model, grid, n_iter=100):
    search = RandomizedSearchCV(
        estimator=model, 
        param_distributions=grid, 
        cv=kf, 
        n_iter=n_iter, 
        n_jobs=-1, 
        random_state=0, 
        verbose=1,
        scoring=rmse_scorer
    )
    return search.fit(X_train, y.values.ravel())

# Define hyperparameter grids
svr_hpg = {
    "C": np.arange(1, 100), 
    "gamma": np.linspace(0.00001, 0.001, 50), 
    "epsilon": np.linspace(0.01, 0.1, 50)
}

gbm_hpg = {
    "max_features": np.linspace(0.2, 0.7, 6), 
    "learning_rate": np.logspace(-3, -1, 100),
    "n_estimators": [100, 200, 500, 1000],
    "max_depth": [3, 5, 7, 9]
}

rfr_hpg = {
    "n_estimators": [100, 300, 500, 1000], 
    "max_features": np.linspace(0.1, 0.5, 5),
    "max_depth": [10, 20, 30, None], 
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

In [ ]:

# Tối ưu RandomForest
print("=== TỐI ƯU RANDOM FOREST ===")
rfc_search = random_search(RandomForestClassifier(), rfr_hpg, n_iter=50)
print(f"Best params: {rfc_search.best_params_}")
print(f"Best score: {-rfc_search.best_score_:.4f}")

# Tối ưu SVR
print("\n=== TỐI ƯU SVR ===")
svc_search = random_search(SVC(probability=True), svr_hpg, n_iter=50)
print(f"Best params: {svc_search.best_params_}")
print(f"Best score: {-svc_search.best_score_:.4f}")

# Tối ưu Gradient Boosting
print("\n=== TỐI ƯU GRADIENT BOOSTING ===")
gbm_search = random_search(GradientBoostingClassifier(), gbm_hpg, n_iter=50)
print(f"Best params: {gbm_search.best_params_}")
print(f"Best score: {-gbm_search.best_score_:.4f}")


=== TỐI ƯU RANDOM FOREST ===
Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [ ]:
rfc_optimized = rfc_search.best_estimator_ 
svc_optimized = svc_search.best_estimator_
gbm_optimized = gbm_search.best_estimator_

## Học tập kết hợp (Ensemble Learning)

**Sử dụng phương pháp stacking(xếp chồng):**

Giảm thiểu Sai số (Tăng Độ chính xác)

Tận dụng Điểm mạnh: Mỗi mô hình hồi quy (như SVR, Random Forest, Gradient Boosting) đều có cách tiếp cận dữ liệu và lỗi khác nhau:

SVR giỏi trong việc tìm ra biên (boundary) và xử lý dữ liệu ở không gian nhiều chiều.

Random Forest giỏi trong việc xử lý dữ liệu phi tuyến tính và chống nhiễu (noise) nhờ vào tính ngẫu nhiên của nó.

Gradient Boosting giỏi trong việc tập trung vào các điểm dữ liệu khó (những nơi mà các mô hình trước đó dự đoán sai).

In [ ]:
base_estimators = [
    ('svr', svc_optimized), 
    ('rfr', rfc_optimized),
    ('gbm', gbm_optimized)
]

In [ ]:
# Mô hình này sẽ học cách kết hợp dự đoán từ 3 mô hình trên.
# RidgeCV hoặc LinearRegression thường là lựa chọn tốt.
meta_model = RidgeCV()

In [ ]:
stacked_model = StackingClassifier(
    estimators=base_estimators,
    final_estimator=meta_model,
    cv=kf,
    n_jobs=-1 
)

## Huấn luyện mô hình

In [ ]:
stacked_model.fit(X_train,y)

c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_stacking.py:970: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


StackingRegressor(cv=KFold(n_splits=5, random_state=0, shuffle=True),
                  estimators=[('svr',
                               SVR(C=np.int64(29),
                                   epsilon=np.float64(0.032040816326530615),
                                   gamma=np.float64(0.0001716326530612245))),
                              ('rfr',
                               RandomForestRegressor(max_features=np.float64(0.2),
                                                     min_samples_split=5,
                                                     n_estimators=1000)),
                              ('gbm',
                               GradientBoostingRegressor(learning_rate=np.float64(0.03944206059437656),
                                                         max_features=np.float64(0.5),
                                                         n_estimators=1000))],
                  final_estimator=RidgeCV(), n_jobs=-1)

In [ ]:
prediction = stacked_model.predict(X_test)
#apply rescale lại cho output đã đc xử lí
prediction = np.exp(prediction)

# Submission

In [ ]:
test = pd.read_csv('../data/test.csv')

In [ ]:
my_prediction = pd.DataFrame({"Id": test['Id'], "SalePrice": prediction})
my_prediction.to_csv('../tmp_sub/exp1.csv',index=False)

# Testing từng model

## Random Forest Regression

In [ ]:
rfc_optimized.fit(X_train,y)
prediction = rfc_optimized.predict(X_test)
rfc_pred = pd.DataFrame({"Id": test['Id'], "SalePrice": np.exp(prediction)})
rfc_pred.to_csv('../submission/rfc_optimized.csv',index=False)

c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


## Support Vector Regression

In [ ]:
svc_optimized.fit(X_train,y)
prediction = svc_optimized.predict(X_test)
svr_pred = pd.DataFrame({"Id": test['Id'], "SalePrice": np.exp(prediction)})
svr_pred.to_csv('../submission/svc_optimized.csv',index=False)

c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Gradient Boosting

In [ ]:
gbm_optimized.fit(X_train,y)
prediction = gbm_optimized.predict(X_test)
gbm_pred = pd.DataFrame({"Id": test['Id'], "SalePrice": np.exp(prediction)})
gbm_pred.to_csv('../submission/gbm_optimized.csv',index=False)

c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
